<a href="https://colab.research.google.com/github/teamgaon/recommendation_algorithms_know/blob/main/220105_sm_nlp_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Classification Knowcode using KoBERT Model

할일 : 텍스트로 변형 가능한 데이터 찾기

예 : 최종학력, 전공, 성별, 등

In [1]:
!pip install gluonnlp
# !pip install mxnet
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'
# !pip install git+https://git@github.com/SKTBrain/KoBERT.git@master
!pip install transformers
!pip install sentencepiece
!pip install kobert-transformers
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-yl9vuuxs/kobert-tokenizer_a478e3b25fec47a982f890319fc018b5
  Running command git clone -q https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-yl9vuuxs/kobert-tokenizer_a478e3b25fec47a982f890319fc018b5
  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-w788inm7
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-w788inm7


In [2]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm.notebook import tqdm
import pandas as pd
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
from kobert_tokenizer import KoBERTTokenizer
from sklearn.model_selection import train_test_split
from google.colab import drive
import matplotlib.pyplot as plt
import warnings

In [3]:
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [18]:
warnings.filterwarnings(action='ignore') 
train1 = pd.read_csv('/content/gdrive/MyDrive/know/train/KNOW_2017.csv', encoding='utf-8')
train2 = pd.read_csv('/content/gdrive/MyDrive/know/train/KNOW_2018.csv', encoding='utf-8')
train3 = pd.read_csv('/content/gdrive/MyDrive/know/train/KNOW_2019.csv', encoding='utf-8')
train4 = pd.read_csv('/content/gdrive/MyDrive/know/train/KNOW_2020.csv', encoding='utf-8')

test1 = pd.read_csv('/content/gdrive/MyDrive/know/test/KNOW_2017_test.csv', encoding='utf-8')
test2 = pd.read_csv('/content/gdrive/MyDrive/know/test/KNOW_2018_test.csv', encoding='utf-8')
test3 = pd.read_csv('/content/gdrive/MyDrive/know/test/KNOW_2019_test.csv', encoding='utf-8')
test4 = pd.read_csv('/content/gdrive/MyDrive/know/test/KNOW_2020_test.csv', encoding='utf-8')

submission = pd.read_csv('/content/gdrive/MyDrive/know/sample_submission.csv', encoding='utf-8')

train2 = train2[train2['idx'] != 18048]
train3 = train3[train3['idx'] != 19871]

In [19]:
#bq4_1a, bq4_1b, bq4_1c, bq5_2, bq19_1, bq30, bq31, bq32, bq33, bq34, bq38_1

cols = ['bq4_1a', 'bq4_1b', 'bq4_1c', 'bq5_2', 'bq19_1', 'bq30', 'bq31', 'bq32', 'bq33', 'bq34', 'bq38_1']

def null_to_no(text : str):
  if text == ' ':
    return '없다'
  else:
    return text

for col in cols:
  train1[col] = train1[col].map(null_to_no)

temp = ''
for col in cols:
  temp += train1[col]
  temp += ' '

train1['text'] = temp

# train1['text'] = train1[cols[0]] + ' ' + train1[cols[1]] + ' ' + train1[cols[2]] + ' ' + train1[cols[3]] + ' ' + train1[cols[4]] + ' ' + train1[cols[5]] + ' ' + train1[cols[6]]

In [20]:
# bq4_1a, bq4_1b, bq4_1c, bq5_2, bq28_1,	bq29,	bq30,	bq31,	bq32, bq33, bq37_1

cols = ['bq4_1a', 'bq4_1b', 'bq4_1c', 'bq5_2', 'bq28_1',	'bq29',	'bq30',	'bq31',	'bq32', 'bq33', 'bq37_1']

def null_to_no(text : str):
  if text == ' ':
    return '없다'
  else:
    return text

for col in cols:
  train2[col] = train2[col].map(null_to_no)

temp = ''
for col in cols:
  temp += train2[col]
  temp += ' '

train2['text'] = temp

# train2['text'] = train2[cols[0]] + ' ' + train2[cols[1]] + ' ' + train2[cols[2]] + ' ' + train2[cols[3]] + ' ' + train2[cols[4]] + ' ' + train2[cols[5]]

In [21]:
# bq4_1a, bq4_1b, bq4_1c, bq5_2, bq18_10,	bq20_1, bq22,	bq23,	bq24, bq27_1

cols = ['bq4_1a', 'bq4_1b', 'bq4_1c', 'bq5_2', 'bq18_10',	'bq20_1', 'bq22',	'bq23',	'bq24', 'bq27_1']

def null_to_no(text : str):
  if text == ' ':
    return '없다'
  else:
    return text

for col in cols:
  train3[col] = train3[col].map(null_to_no)

temp = ''
for col in cols:
  temp += train3[col]
  temp += ' '

train3['text'] = temp
# train3['text'] = train3[cols[0]] + ' ' + train3[cols[1]] + ' ' + train3[cols[2]] + ' ' + train3[cols[3]] + ' ' + train3[cols[4]] + ' ' + train3[cols[5]]

In [22]:
# bq4_1a, bq4_1b, bq4_1c, bq5_2, bq18_10, bq20_1, bq26_1

cols = ['bq4_1a', 'bq4_1b', 'bq4_1c', 'bq5_2', 'bq18_10', 'bq20_1', 'bq26_1']

def null_to_no(text : str):
  if text == ' ':
    return '없다'
  else:
    return text

for col in cols:
  train4[col] = train4[col].map(null_to_no)

temp = ''
for col in cols:
  temp += train4[col]
  temp += ' '

train4['text'] = temp

# train4['text'] = train4[cols[0]] + ' ' + train4[cols[1]] + ' ' + train4[cols[2]]

In [23]:
df = pd.concat([train1[['idx', 'text', 'knowcode']], train2[['idx', 'text', 'knowcode']], train3[['idx', 'text', 'knowcode']], train4[['idx', 'text', 'knowcode']]], axis=0)
df = df.reset_index(drop=True)
df

,idx,text,knowcode
0,0,자동차도장기능사 없다 없다 실무교육 생산설비의 자동화로 없다 없다 없다 건설현장 노...,825101
1,1,"건축전기설비기술사 없다 없다 없다 건설 수주가 없어서 없다 매타기, 드라이버, 가위...",140204
2,2,"건축전기설비기술사 없다 없다 없다 신축 건설경기가 좋지 않아서 없다 건축설계표, 오...",140204
3,3,"지적기사 없다 없다 측량 및 정보통신기술 교육 업무량 감소, 자동화로 인해서 지적기...",140601
4,4,건축전기설비기술사 없다 없다 없다 부동산 거품 빠지면서 건축 경기도 덩달아 불황 없...,140204
...,...,...,...
35228,35244,없다 없다 없다 없다 없다 건설 경기의 어려움으로 다소 어려움 지구과학,121105
35229,35245,없다 없다 없다 없다 없다 광고 홍보 관련 시장이 켜져서 광고학,26102
35230,35246,보험계리사 없다 없다 없다 없다 보험은 계속 필요 할 것이다 국문,31301
35231,35247,화학분석기능사 없다 없다 없다 없다 인구수명이 길어지므로서 신약개발은 갈수록 많아질...,154105


In [24]:
def remove_ddaom(text:str):
  return str(text).replace('"', '')

df['text'] = df['text'].map(remove_ddaom)

def remove_enter(text:str):
  return text.replace('\n', ' ')

df['text'] = df['text'].map(remove_enter)

In [25]:
## CPU
# device = torch.device("cpu")

## GPU
device = torch.device("cuda:0")

In [26]:
bertmodel, vocab = get_pytorch_kobert_model(cachedir=".cache")

using cached model. /content/.cache/kobert_v1.zip
using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [27]:
my_dict = {}
for i in range(len(df['knowcode'].unique())):
  my_dict[df['knowcode'].unique()[i]] = i

In [28]:
def target_to_label(text:str):
  return my_dict[text]

df['target_label'] = df['knowcode'].map(target_to_label)

In [29]:
dataset_train, dataset_test = train_test_split(df[['idx', 'text', 'target_label']], test_size=0.2, random_state=42, shuffle=True)
dataset_train.to_csv('dataset_train.txt', index=False, header=True, sep='\t', encoding='utf8')
dataset_test.to_csv('dataset_test.txt', index=False, header=True, sep='\t', encoding='utf8')

In [30]:
dataset_train = nlp.data.TSVDataset('/content/dataset_train.txt', field_indices=[1,2], num_discard_samples=1)
dataset_test = nlp.data.TSVDataset('/content/dataset_test.txt', field_indices=[1,2], num_discard_samples=1)

In [31]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [32]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))


In [33]:
## Setting parameters
max_len = 256
batch_size = 8
warmup_ratio = 0.1
num_epochs = 20
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [34]:
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

In [35]:
next(iter(data_train))

(array([   2, 2872, 6516, 7350, 6361, 5478, 6493, 3273, 3273, 3273, 3697,
        5712, 5859, 6410, 3817, 4930, 2588,  517, 5366, 6630,  529, 5712,
        6255, 6150, 7096, 6198, 4760, 6493, 7784, 5550, 3803, 7003, 7342,
        7078, 2339, 7850, 3273, 1788, 6084,  517,   46, 2872, 7350, 7479,
        7227, 5561,  517,   46, 3563, 7461, 7202, 4297, 7706, 4898, 2272,
        5503,  285,  278,  517,   46, 1055, 3273, 3273, 5137, 5452, 7821,
           3,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1, 

In [36]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=4)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=4)

In [37]:
len(df['target_label'].unique())

585

In [38]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=585,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [39]:
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

In [40]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [41]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

In [42]:
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

In [43]:
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [44]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [ ]:
train = []
test = []
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in tqdm(enumerate(train_dataloader), total=len(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    train.append(train_acc / (batch_id+1))
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in tqdm(enumerate(test_dataloader), total=len(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))
    test.append(test_acc / (batch_id+1))
    # torch.cuda.empty_cache()

  0%|          | 0/3524 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 6.231141090393066 train acc 0.125
epoch 1 batch id 201 loss 6.563636779785156 train acc 0.0018656716417910447
epoch 1 batch id 401 loss 6.555638790130615 train acc 0.0021820448877805485
epoch 1 batch id 601 loss 6.395702362060547 train acc 0.0020798668885191347
epoch 1 batch id 801 loss 6.27824592590332 train acc 0.002184769038701623
epoch 1 batch id 1001 loss 6.368040084838867 train acc 0.0037462537462537465
epoch 1 batch id 1201 loss 6.368524551391602 train acc 0.004995836802664446
epoch 1 batch id 1401 loss 6.405701637268066 train acc 0.005620985010706638


In [ ]:
torch.save(model, '/content/gdrive/MyDrive/know/model2.pt')

In [ ]:
#bq4_1a, bq4_1b, bq4_1c, bq5_2, bq19_1, bq30, bq31, bq32, bq33, bq34, bq38_1

cols = ['bq4_1a', 'bq4_1b', 'bq4_1c', 'bq5_2', 'bq19_1', 'bq30', 'bq31', 'bq32', 'bq33', 'bq34', 'bq38_1']

def null_to_no(text : str):
  if text == ' ':
    return '없다'
  else:
    return text

for col in cols:
  test1[col] = test1[col].map(null_to_no)

temp = ''
for col in cols:
  temp += test1[col]
  temp += ' '

test1['text'] = temp

# test1['text'] = test1[cols[0]] + ' ' + test1[cols[1]] + ' ' + test1[cols[2]] + ' ' + test1[cols[3]] + ' ' + test1[cols[4]] + ' ' + test1[cols[5]] + ' ' + test1[cols[6]]

In [ ]:
# bq4_1a, bq4_1b, bq4_1c, bq5_2, bq28_1,	bq29,	bq30,	bq31,	bq32, bq33, bq37_1

cols = ['bq4_1a', 'bq4_1b', 'bq4_1c', 'bq5_2', 'bq28_1',	'bq29',	'bq30',	'bq31',	'bq32', 'bq33', 'bq37_1']

def null_to_no(text : str):
  if text == ' ':
    return '없다'
  else:
    return text

for col in cols:
  test2[col] = test2[col].map(null_to_no)

temp = ''
for col in cols:
  temp += test2[col]
  temp += ' '

test2['text'] = temp

# test2['text'] = test2[cols[0]] + ' ' + test2[cols[1]] + ' ' + test2[cols[2]] + ' ' + test2[cols[3]] + ' ' + test2[cols[4]] + ' ' + test2[cols[5]]

In [ ]:
# bq4_1a, bq4_1b, bq4_1c, bq5_2, bq18_10,	bq20_1, bq22,	bq23,	bq24, bq27_1

cols = ['bq4_1a', 'bq4_1b', 'bq4_1c', 'bq5_2', 'bq18_10',	'bq20_1', 'bq22',	'bq23',	'bq24', 'bq27_1']

def null_to_no(text : str):
  if text == ' ':
    return '없다'
  else:
    return text

for col in cols:
  test3[col] = test3[col].map(null_to_no)

temp = ''
for col in cols:
  temp += test3[col]
  temp += ' '

test3['text'] = temp
# test3['text'] = test3[cols[0]] + ' ' + test3[cols[1]] + ' ' + test3[cols[2]] + ' ' + test3[cols[3]] + ' ' + test3[cols[4]] + ' ' + test3[cols[5]]

In [ ]:
# bq4_1a, bq4_1b, bq4_1c, bq5_2, bq18_10, bq20_1, bq26_1

cols = ['bq4_1a', 'bq4_1b', 'bq4_1c', 'bq5_2', 'bq18_10', 'bq20_1', 'bq26_1']

def null_to_no(text : str):
  if text == ' ':
    return '없다'
  else:
    return text

for col in cols:
  test4[col] = test4[col].map(null_to_no)

temp = ''
for col in cols:
  temp += test4[col]
  temp += ' '

test4['text'] = temp

# test4['text'] = test4[cols[0]] + ' ' + test4[cols[1]] + ' ' + test4[cols[2]]

In [ ]:
df = pd.concat([test1[['idx', 'text', 'knowcode']], test2[['idx', 'text', 'knowcode']], test3[['idx', 'text', 'knowcode']], test4[['idx', 'text', 'knowcode']]], axis=0)
df = df.reset_index(drop=True)
df

,idx,text,knowcode
0,0,생산설비의 자동화로 없다 없다 없다 건설현장 노무직 없다 실업,825101
1,1,"건설 수주가 없어서 없다 매타기, 드라이버, 가위, CAD 없다 없다 없다 건축공학",140204
2,2,"신축 건설경기가 좋지 않아서 없다 건축설계표, 오토캐드, 파이프, 보일러 없다 없다...",140204
3,3,"업무량 감소, 자동화로 인해서 지적기사 토탈스테이션, SZP 무 지적공무원 없다 환경학과",140601
4,4,"부동산 거품 빠지면서 건축 경기도 덩달아 불황 없다 캐드, 엑셀, 건축도면 없다 없...",140204
...,...,...,...
35229,35244,없다 건설 경기의 어려움으로 다소 어려움 지구과학,121105
35230,35245,없다 광고 홍보 관련 시장이 켜져서 광고학,26102
35231,35246,없다 보험은 계속 필요 할 것이다 국문,31301
35232,35247,없다 인구수명이 길어지므로서 신약개발은 갈수록 많아질 것이라서 화학공학,154105


In [ ]:
def remove_ddaom(text:str):
  return str(text).replace('"', '')

df['text'] = df['text'].map(remove_ddaom)

def remove_enter(text:str):
  return text.replace('\n', ' ')

df['text'] = df['text'].map(remove_enter)

In [ ]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

def predict(sentence):
  data = [sentence, '0']
  dataset_another = [data]
  logits = 0
  another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
  test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=2)
  
  model.eval()

  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
      token_ids = token_ids.long().to(device)
      segment_ids = segment_ids.long().to(device)

      valid_length= valid_length
      label = label.long().to(device)

      out = model(token_ids, valid_length, segment_ids)

      for i in out:
        logits = i
        logits = logits.detach().cpu().numpy()
        logits = np.argmax(logits)

  return logits

label_answer = []
step = 0
for i in tqdm(df['text']):
  label_answer.append(predict(i))

In [ ]:
submission = pd.read_csv('/content/gdrive/MyDrive/know/sample_submission.csv') # sample submission 불러오기
submission['knowcode'] = label_answer
submission

In [ ]:
submission.to_csv('/content/gdrive/MyDrive/know/submission.csv', index=False, encoding='utf8')